In [ ]:
from hdf5_loader import extract_arrays_metadata, extract_array

import numpy as np

In [ ]:
arrays_metadata = extract_arrays_metadata('dam_dataset_long_big/dam3Bumps.h5')

mem_vertices = extract_array(arrays_metadata, 'nodesSpatial_Domain0')
vertices = np.array(mem_vertices[:, 0:2])

indices = extract_array(arrays_metadata, 'elementsSpatial_Domain0')

h = extract_array(arrays_metadata, 'h_t0')
bathymetry = extract_array(arrays_metadata, 'bathymetry0_t0')

In [ ]:
from ipywidgets import Image

In [ ]:
texture = Image.from_file('tex.jpeg')

In [ ]:
texture

In [ ]:
from ipywidgets import Play, IntSlider, HBox, link
from ipygany import Scene, Data, Component, PolyMesh, Water, UnderWater, Data, Component, Threshold, Warp

In [ ]:
warp_value = 3

In [ ]:
h = extract_array(arrays_metadata, 'h_t0')

h_component = Component(name='h', array=h)

z_water = h + bathymetry

water_mesh = PolyMesh(
    vertices=np.append(vertices, z_water.reshape((z_water.shape[0], 1)) * warp_value, axis=1),
    triangle_indices=indices,
    data=[Data(name='h', components=[h_component])]
)

actual_water = Threshold(water_mesh, input='h', min=0.00000001, max=1000)

floor = PolyMesh(
    vertices=np.append(vertices, bathymetry.reshape((bathymetry.shape[0], 1)) * warp_value, axis=1),
    triangle_indices=indices,
    data=[Data(name='underwater', components=[h_component])]
)

water = Water(
    actual_water, 
    under_water_blocks=(UnderWater(floor), ),
    caustics_enabled=True,
)

scene = Scene((water, ))

def update_step(change):
    i = change['new']
    
    h = extract_array(arrays_metadata, 'h_t{}'.format(i))

    h_component.array = h

    z_water = h + bathymetry

    water_mesh.vertices = np.append(vertices, z_water.reshape((z_water.shape[0], 1)) * warp_value, axis=1)

play = Play(description='Step:', min=0, max=200, value=0, interval=50)
play.observe(update_step, names=['value'])

progress = IntSlider(value=0, step=1, min=0, max=200)
link((progress, 'value'), (play, 'value'))

display(HBox((play, progress)))

scene

In [ ]:
scene.background_color = 'aliceblue'

In [ ]:
water.under_water_blocks[0].default_color = 'rgb(237, 201, 175)'

In [ ]:
water.under_water_blocks[0].texture = texture

In [ ]:
water.under_water_blocks[0].texture = None

In [ ]:
water.caustics_factor = 0.1

In [ ]:
from ipywebrtc import VideoRecorder, WidgetStream

In [ ]:
widget_stream = WidgetStream(widget=scene)
widget_stream

In [ ]:
image_recorder = VideoRecorder(stream=widget_stream)
image_recorder